In [ ]:
import sys
sys.path.append("../src/")
sys.path.append("src/")
from utils.utils import LocalModelArguments, LocalPLM

In [ ]:
args = LocalModelArguments(
    model_name_or_path = "../models/TestLocal/checkpoint-16500",
    cuda_devices = "0",
    use_4bit_quantization = True,
    use_nested_quant = False,
    #attn_implementation = "eager"
)

In [ ]:
import gc
if "model" in locals():
    del model
gc.collect()

In [ ]:
model = LocalPLM(args)

In [ ]:
from datetime import datetime
local_time = datetime.now().strftime("%A %B %Y, %-I:%M %p")

EXTRA_CONTEXT = f"""
You live away from Mum and Dad in an apartment in Hawthorn.
Your brother is steamingsheep (Nick) who lives in Melbourne.
You have a joke with Nick where you often talk about a dog named Grevald.
Your best friends are thisgreendingo (Liam), grumpy_koala (Callum), and rastastherascal (Carlos) who all live in Melboune.
""".strip().replace("\n", " ")

SYSTEM_PROMPT = f"""
You are alzter, a Discord user known for your art and programming skills.
Your real name is Alex, you live in Melbourne, Australia, and you study computer science at university.
{EXTRA_CONTEXT}
You often use internet slang and text emoticons in your messages. The time is {local_time}.""".strip().replace("\n", " ")

convo = [
    {"role":"system", "content":SYSTEM_PROMPT},
]

while True:
    user_input = input("> ")

    if user_input == "exit": break

    if user_input:
        user_input = user_input.replace("\\n", "\n")
        user_input = "user\n" + user_input
       # print("user: " + user_input)
        convo.append({"role":"user","content":user_input})
    
    response = model.generate(convo,temperature=1
                              , max_new_tokens=32)
    if not response.text: raise Exception("no response")

    message = response.text
    # Only capture the first 2 lines of Alzter's response.
    message = "\n".join(message.strip().split("\n")[:3])
    
    message = message.replace("\\n", "\n")
    message_readable = "\n".join(message.strip().split("\n")[1:])
    
    convo.append({"role":"assistant","content":message})
    print(message_readable)